In [5]:
%pip install fasttext nltk


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import nltk
# Configure SSL to bypass certificate verification
import ssl
import urllib.request

ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

urllib.request.install_opener(urllib.request.build_opener(urllib.request.HTTPSHandler(context=ssl_context)))

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chakrabortyb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/chakrabortyb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/chakrabortyb/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
import urllib.request
import zipfile
import os

datasets_dir = "."

# download text8
url = "http://mattmahoney.net/dc/text8.zip"
zip_path = os.path.join(datasets_dir, "text8.zip")
text8_path = os.path.join(datasets_dir, "text8")

if not os.path.exists(text8_path):
    urllib.request.urlretrieve(url, zip_path)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(datasets_dir)

In [7]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

with open(text8_path, 'r') as f:
    text8_data = f.read()

# remove stop words
stop_words = set(stopwords.words('english'))
text8_tokens = word_tokenize(text8_data)
text8_filtered = [w for w in text8_tokens if w not in stop_words]

# create a smaller subset (200k characters)
text8_200k_path = os.path.join(datasets_dir, "text8_200k.txt")
text8_200k_data = text8_filtered[:200000]

if not os.path.exists(text8_200k_path):
    with open(text8_200k_path, 'w') as f:
        for token in text8_200k_data:
            f.write(token + "\n")

In [8]:
# process the full text8 dataset
text8_full_path = os.path.join(datasets_dir, "text8.txt")

if not os.path.exists(text8_full_path):
    with open(text8_full_path, 'w') as f:
        for token in text8_filtered:
            f.write(token + "\n")

In [9]:
import fasttext
import os

model = fasttext.train_unsupervised(
    'text8_200k.txt',       # Input file
    model='skipgram',       
    dim=50,                 # Embedding dimension
    epoch=10,               # Number of training iterations
    minCount=1,             # Minimum word frequency
    minn=3,                 # Minimum character n-gram length
    maxn=6                  # Maximum character n-gram length
)

embeddings_path = os.path.join("..", "embeddings", "fasttext_embeddings.bin")
model.save_model(embeddings_path)

Read 0M words
Number of words:  25110
Number of labels: 0
Progress: 100.0% words/sec/thread: 2539731 lr:  0.000000 avg.loss:  3.528255 ETA:   0h 0m 0s


In [10]:
model = fasttext.train_unsupervised(
    'text8.txt',            # Input file
    model='skipgram',       
    dim=50,                 # Embedding dimension
    epoch=10,               # Number of training iterations
    minCount=1,             # Minimum word frequency
    minn=3,                 # Minimum character n-gram length
    maxn=6                  # Maximum character n-gram length
)

embeddings_path = os.path.join("..", "embeddings", "fasttext_full_embeddings.bin")
model.save_model(embeddings_path)

Read 21M words
Number of words:  253698
Number of labels: 0
Progress: 100.0% words/sec/thread: 5228054 lr:  0.000000 avg.loss:  1.523068 ETA:   0h 0m 0s


In [11]:
def get_word_embeddings(model):
    king_vector = model.get_word_vector('king')
    print(f"Vector for 'king': {king_vector[:5]}...")
    print(f"Vector shape: {king_vector.shape}")
    
    kingdom_vector = model.get_word_vector('kingdom')
    print(f"Vector for 'kingdom' (OOV): {kingdom_vector[:5]}...")
    
    return king_vector, kingdom_vector

king_vec, kingdom_vec = get_word_embeddings(model)

Vector for 'king': [-3.831587   2.5720918 -2.0376558  1.4025882  4.792776 ]...
Vector shape: (50,)
Vector for 'kingdom' (OOV): [-1.261026    0.8239888  -0.65587074  0.51792884  1.6317406 ]...
